## Import stuff

In [7]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import torch 
import torch.jit
import torch.nn as nn 
import torchvision
from torchvision import transforms, datasets
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import torch 
import torch.jit
import torch.nn as nn 
import torchvision
from torchvision import transforms, datasets
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn import datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
# from sklearn.linear_model import 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import ClusterCentroids
import math
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

print(tf.__version__)

from tensorflow.python.client import device_lib
print("tensorflow doesn't detects GPU: ")
print(device_lib.list_local_devices() )
print(tf.test.gpu_device_name())
print(tf.config.list_physical_devices())

print("torch detects GPU: " , torch.cuda.is_available())



2.3.0
tensorflow doesn't detects GPU: 
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18265888575262745852
]

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
torch detects GPU:  True


# last value known for each patient + ratio between first and last + other features

In [19]:
for file in ["train", "test"]:
    df = pd.read_csv(file+"_features.csv")
    orig_ordering = pd.DataFrame(np.array(df.iloc[:, :].values,  dtype=float)[11::12], columns=df.columns[:])

    #RATIO
    ratios = pd.DataFrame(df.groupby(["pid"]).last()/df.groupby(["pid"]).first()-1)
    ratios = pd.merge(orig_ordering["pid"], ratios, on="pid")
    ratios = ratios.drop(["Time", "Age"], axis=1)
    ratios.columns = [x+"_ratio" for x in ratios.columns]
    ratios.rename(columns={'pid_ratio': 'pid'}, inplace=True)

    #DERIVATIVE
    derivative = pd.DataFrame(df.groupby(["pid"]).last()-df.groupby(["pid"]).first())
    derivative = pd.merge(orig_ordering["pid"], derivative, on="pid")
    derivative = derivative.drop(["Time", "Age"], axis=1)
    derivative.columns = [x+"_derivative" for x in derivative.columns]
    derivative.rename(columns={'pid_derivative': 'pid'}, inplace=True)

    #MIN
    min = pd.DataFrame(df.groupby(["pid"]).min())
    min = pd.merge(orig_ordering["pid"], min, on="pid")
    min = min.drop(["Time", "Age"], axis=1)
    min.columns = [x+"_min" for x in min.columns]
    min.rename(columns={'pid_min': 'pid'}, inplace=True)

    #MAX
    max = pd.DataFrame(df.groupby(["pid"]).max())
    max = pd.merge(orig_ordering["pid"], max, on="pid")
    max = max.drop(["Time", "Age"], axis=1)
    max.columns = [x+"_max" for x in max.columns]
    max.rename(columns={'pid_max': 'pid'}, inplace=True)

    #COUNT
    count = pd.DataFrame(df.groupby(["pid"]).count())
    count = pd.merge(orig_ordering["pid"], count, on="pid")
    count = count.drop(["Time", "Age"], axis=1)
    count.columns = [x+"_count" for x in count.columns]
    count.rename(columns={'pid_count': 'pid'}, inplace=True)

    #MEAN
    mean = pd.DataFrame(df.groupby(["pid"]).mean())
    mean = pd.merge(orig_ordering["pid"], mean, on="pid")
    mean = mean.drop(["Time", "Age"], axis=1)
    mean.columns = [x+"_mean" for x in mean.columns]
    mean.rename(columns={'pid_mean': 'pid'}, inplace=True)

    #MEDIAN
    median = pd.DataFrame(df.groupby(["pid"]).median())
    median = pd.merge(orig_ordering["pid"], median, on="pid")
    median = median.drop(["Time", "Age"], axis=1)
    median.columns = [x+"_median" for x in median.columns]
    median.rename(columns={'pid_median': 'pid'}, inplace=True)

    #VAR
    var = pd.DataFrame(df.groupby(["pid"]).var())
    var = pd.merge(orig_ordering["pid"], var, on="pid")
    var = var.drop(["Time", "Age"], axis=1)
    var.columns = [x+"_var" for x in var.columns]
    var.rename(columns={'pid_var': 'pid'}, inplace=True)

    #LAST HOUR
    last_hour = pd.DataFrame(df.groupby(["pid"]).last())
    last_hour = pd.merge(orig_ordering["pid"], last_hour, on="pid")
    last_hour.columns = [x+"_last_hour" for x in last_hour.columns]
    last_hour.rename(columns={'pid_last_hour': 'pid'}, inplace=True)
    last_hour.rename(columns={'Time_last_hour': 'Time'}, inplace=True)
    last_hour.rename(columns={'Age_last_hour': 'Age'}, inplace=True)

    #SQUARE
    square = last_hour**2
    square = square.drop(["Time", "Age"], axis=1)
    square.columns = [x+"_square" for x in square.columns]
    square.rename(columns={'pid_square': 'pid'}, inplace=True)
    square["pid"] = orig_ordering["pid"]

    #LOG
    log = np.log(last_hour)
    log = log.drop(["Time", "Age"], axis=1)
    log.columns = [x+"_log" for x in log.columns]
    log.rename(columns={'pid_log': 'pid'}, inplace=True)
    log["pid"] = orig_ordering["pid"]


    # merge the dataframes
    # df = pd.concat([last_hour, ratios, min, max,square,log,count], axis=1)
    # df = pd.concat([last_hour, ratios, min, max,square,log,count,mean], axis=1)
    # df = pd.concat([last_hour, ratios, min, max,square,log,count,median,var], axis=1)
    df = pd.concat([last_hour, ratios, min, max,square,log,count,mean,median,var,derivative], axis=1)


    #make "pid" the first column
    first_column = df.pop('pid')
    df.insert(0, 'pid', orig_ordering["pid"])

    df.replace([np.inf, -np.inf], 0, inplace=True)
    print(df.shape)
    df.to_csv(file+'_features_ratio.csv', index=False, float_format='%.3f',na_rep='nan')


(18995, 377)
(12664, 377)


# imputation

In [20]:
for file in ["train", "test"]:
    df = pd.read_csv(file+"_features_ratio.csv")
    orig_ordering = pd.DataFrame(np.array(df.iloc[:, :].values,  dtype=float)[:], columns=df.columns[:])
    X = np.array(df.iloc[:, 1:].values,  dtype=float)

    imputer = KNNImputer(n_neighbors=5)
    # imputer = SimpleImputer(strategy='median')
    df_imputed = imputer.fit_transform(X)
    df_imputed = pd.DataFrame(df_imputed)
    df_imputed.insert(0, 'pid', orig_ordering["pid"])
    df_imputed.columns = df.columns
    df_imputed.to_csv(file+'_features_imputed.csv', index=False, float_format='%.3f',na_rep='nan')


# normalization

In [22]:
for file in ["train", "test"]:
    df = pd.read_csv(file+"_features_imputed.csv")
    orig_ordering = pd.DataFrame(np.array(df.iloc[:, :].values,  dtype=float)[:], columns=df.columns[:])
    X = np.array(df.iloc[:, 1:].values,  dtype=float)

    # df2 = df.iloc[:,1:]
    # # df2 = df2.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
    # df2 = df2.apply(lambda x: (x - x.mean()) / x.std())
    # df.iloc[:,1:] = df2

    # scaler = StandardScaler()
    scaler = MinMaxScaler()
    df_scaled = scaler.fit_transform(X)
    df_scaled = pd.DataFrame(df_scaled)
    df_scaled.insert(0, 'pid', orig_ordering["pid"])
    df_scaled.columns = df.columns
    df_scaled.to_csv(file+'_features_imputed_normalized.csv', index=False, float_format='%.3f',na_rep='nan')












# EVERYTHING BELOW IS EXPERIMENTAL AND NOT NEEDED

In [11]:
df = pd.read_csv("train_features.csv")
orig_ordering = pd.DataFrame(np.array(df.iloc[:, :].values,  dtype=float)[11::12], columns=df.columns[:])
ratios = pd.DataFrame(df.groupby(["pid"]).last()-df.groupby(["pid"]).first())
ratios = pd.merge(orig_ordering["pid"], ratios, on="pid")
ratios = ratios.drop(["Time", "Age"], axis=1)
ratios.columns = [x+"_ratio" for x in ratios.columns]
ratios.rename(columns={'pid_ratio': 'pid'}, inplace=True)
ratios

,pid,EtCO2_ratio,PTT_ratio,BUN_ratio,Lactate_ratio,Temp_ratio,Hgb_ratio,HCO3_ratio,BaseExcess_ratio,RRate_ratio,...,Alkalinephos_ratio,SpO2_ratio,Bilirubin_direct_ratio,Chloride_ratio,Hct_ratio,Heartrate_ratio,Bilirubin_total_ratio,TroponinI_ratio,ABPs_ratio,pH_ratio
0,1.0,NaN,NaN,0.0,NaN,2.0,-0.2,2.0,2.0,2.0,...,NaN,0.0,NaN,-3.0,-2.4,-25.0,NaN,NaN,-40.0,0.06
1,10.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,-7.0,...,0.0,-1.0,NaN,NaN,0.0,-7.0,0.0,0.00,-9.0,NaN
2,100.0,NaN,0.0,0.0,NaN,3.0,0.0,0.0,NaN,-1.0,...,NaN,-3.0,NaN,0.0,0.0,19.0,NaN,NaN,-42.0,NaN
3,1000.0,3.0,NaN,0.0,-0.39,2.0,0.0,NaN,NaN,0.0,...,NaN,-5.0,NaN,NaN,0.0,-31.0,NaN,NaN,1.0,0.00
4,10000.0,NaN,5.7,0.0,NaN,1.0,-0.6,1.0,2.0,-3.0,...,NaN,-1.0,NaN,-1.0,-2.0,26.0,NaN,NaN,22.0,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18990,9993.0,NaN,NaN,1.0,NaN,-1.0,1.9,NaN,NaN,-7.0,...,-2.0,5.0,NaN,NaN,4.3,-22.0,-0.1,2.40,-18.0,NaN
18991,9995.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,NaN,4.0,...,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,-1.0,NaN
18992,9996.0,NaN,NaN,NaN,NaN,1.0,-0.2,NaN,0.0,8.0,...,NaN,0.0,NaN,NaN,-7.9,9.0,NaN,NaN,-18.0,0.02
18993,9998.0,NaN,0.0,0.0,NaN,1.0,0.0,NaN,NaN,1.0,...,NaN,0.0,NaN,NaN,0.0,-6.0,NaN,-1.31,47.0,NaN


In [18]:
df = pd.read_csv("train_features.csv")
orig_ordering = pd.DataFrame(np.array(df.iloc[:, :].values,  dtype=float)[11::12], columns=df.columns[:])
last_hour = pd.DataFrame(df.groupby(["pid"]).last())
last_hour = pd.merge(orig_ordering["pid"], last_hour, on="pid")
last_hour.columns = [x+"_last_hour" for x in last_hour.columns]
last_hour.rename(columns={'pid_last_hour': 'pid'}, inplace=True)
last_hour.rename(columns={'Time_last_hour': 'Time'}, inplace=True)
last_hour.rename(columns={'Age_last_hour': 'Age'}, inplace=True)
last_hour

,pid,Time,Age,EtCO2_last_hour,PTT_last_hour,BUN_last_hour,Lactate_last_hour,Temp_last_hour,Hgb_last_hour,HCO3_last_hour,...,Alkalinephos_last_hour,SpO2_last_hour,Bilirubin_direct_last_hour,Chloride_last_hour,Hct_last_hour,Heartrate_last_hour,Bilirubin_total_last_hour,TroponinI_last_hour,ABPs_last_hour,pH_last_hour
0,1.0,14,34.0,NaN,NaN,12.0,NaN,38.0,8.5,26.0,...,NaN,100.0,NaN,111.0,22.2,69.0,NaN,NaN,102.0,7.39
1,10.0,12,71.0,NaN,27.8,12.0,NaN,36.0,14.6,NaN,...,68.0,98.0,NaN,NaN,42.1,75.0,1.3,0.01,136.0,NaN
2,100.0,13,68.0,NaN,20.9,21.0,NaN,38.0,12.5,27.0,...,NaN,97.0,NaN,101.0,36.8,113.0,NaN,NaN,116.0,NaN
3,1000.0,12,79.0,29.0,NaN,22.0,3.66,38.0,9.2,NaN,...,NaN,95.0,NaN,NaN,27.3,79.0,NaN,NaN,125.0,7.30
4,10000.0,12,76.0,NaN,31.4,22.0,NaN,37.0,10.4,26.0,...,NaN,99.0,NaN,103.0,29.3,90.0,NaN,NaN,132.0,7.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18990,9993.0,12,80.0,NaN,NaN,14.0,NaN,35.0,9.3,NaN,...,64.0,100.0,NaN,NaN,28.3,112.0,0.5,11.88,101.0,NaN
18991,9995.0,12,73.0,NaN,55.5,50.0,NaN,36.0,11.2,29.0,...,NaN,96.0,NaN,89.0,35.6,64.0,NaN,NaN,174.0,NaN
18992,9996.0,18,53.0,NaN,NaN,NaN,NaN,38.0,10.7,NaN,...,NaN,100.0,NaN,NaN,25.1,95.0,NaN,NaN,95.0,7.43
18993,9998.0,12,89.0,NaN,34.7,13.0,NaN,37.0,8.4,NaN,...,NaN,100.0,NaN,NaN,25.6,100.0,NaN,0.85,156.0,NaN


In [56]:
df = pd.read_csv("train_features.csv")
count = pd.DataFrame(df.groupby(["pid"]).count())
count

,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
pid,,,,,,,,,,,,,,,,,,,,,
1,12,12,0,0,3,0,8,3,3,6,...,0,12,0,3,6,12,0,0,12,7
2,12,12,0,1,1,0,3,1,0,0,...,0,11,0,0,1,11,0,1,11,0
4,12,12,0,1,2,0,3,2,0,0,...,1,11,1,0,2,11,1,1,11,0
6,12,12,0,1,2,2,12,6,2,7,...,0,12,0,2,10,12,0,0,12,7
8,12,12,0,0,1,0,3,0,0,0,...,0,10,0,0,0,11,0,1,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31653,12,12,0,1,3,4,2,3,3,2,...,0,11,0,3,5,11,0,0,6,2
31654,12,12,0,0,2,0,2,1,0,0,...,0,11,0,0,1,11,0,0,11,0
31656,12,12,0,0,1,0,3,1,1,4,...,0,10,0,1,1,10,0,0,10,4


In [64]:
df = pd.read_csv("train_features.csv")
ratios = pd.DataFrame(df.groupby(["pid"]).last()/df.groupby(["pid"]).first()-1)
orig_ordering = pd.DataFrame(np.array(df.iloc[:, :].values,  dtype=float)[11::12], columns=df.columns[:])
ratios = pd.merge(orig_ordering["pid"], ratios, on="pid")
ratios = ratios.drop(["Time", "Age"], axis=1)
ratios.columns = [x+"_ratio" for x in ratios.columns]
ratios.rename(columns={'pid_ratio': 'pid'}, inplace=True)
ratios

,pid,EtCO2_ratio,PTT_ratio,BUN_ratio,Lactate_ratio,Temp_ratio,Hgb_ratio,HCO3_ratio,BaseExcess_ratio,RRate_ratio,...,Alkalinephos_ratio,SpO2_ratio,Bilirubin_direct_ratio,Chloride_ratio,Hct_ratio,Heartrate_ratio,Bilirubin_total_ratio,TroponinI_ratio,ABPs_ratio,pH_ratio
0,1.0,NaN,NaN,0.000000,NaN,0.055556,-0.022989,0.083333,-1.0,0.125000,...,NaN,0.000000,NaN,-0.026316,-0.097561,-0.265957,NaN,NaN,-0.281690,0.008186
1,10.0,NaN,0.00000,0.000000,NaN,0.000000,0.000000,NaN,NaN,-0.304348,...,0.000000,-0.010101,NaN,NaN,0.000000,-0.085366,0.000000,0.000000,-0.062069,NaN
2,100.0,NaN,0.00000,0.000000,NaN,0.085714,0.000000,0.000000,NaN,-0.071429,...,NaN,-0.030000,NaN,0.000000,0.000000,0.202128,NaN,NaN,-0.265823,NaN
3,1000.0,0.115385,NaN,0.000000,-0.096296,0.055556,0.000000,NaN,NaN,0.000000,...,NaN,-0.050000,NaN,NaN,0.000000,-0.281818,NaN,NaN,0.008065,0.000000
4,10000.0,NaN,0.22179,0.000000,NaN,0.027778,-0.054545,0.040000,inf,-0.272727,...,NaN,-0.010000,NaN,-0.009615,-0.063898,0.406250,NaN,NaN,0.200000,0.006803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18990,9993.0,NaN,NaN,0.076923,NaN,-0.027778,0.256757,NaN,NaN,-0.280000,...,-0.030303,0.052632,NaN,NaN,0.179167,-0.164179,-0.166667,0.253165,-0.151261,NaN
18991,9995.0,NaN,0.00000,0.000000,NaN,0.000000,0.000000,0.000000,NaN,0.235294,...,NaN,0.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN,-0.005714,NaN
18992,9996.0,NaN,NaN,NaN,NaN,0.027027,-0.018349,NaN,NaN,0.800000,...,NaN,0.000000,NaN,NaN,-0.239394,0.104651,NaN,NaN,-0.159292,0.002699
18993,9998.0,NaN,0.00000,0.000000,NaN,0.027778,0.000000,NaN,NaN,0.052632,...,NaN,0.000000,NaN,NaN,0.000000,-0.056604,NaN,-0.606481,0.431193,NaN


# very old stuff

In [14]:
df = pd.read_csv("train_features.csv")
# patients = pd.read_csv("test_features.csv")

# go through 12 rows at a time
for i in range(0, len(df), 12):
    # get the 12 rows
    patient = df.iloc[i:i+12]
    #fill each column with the last value
    for col in patient.columns:
        mean = patient[col].mean()
        if not math.isnan(mean):
            # print(col)
            # print(patient[col].dropna().iloc[-1])
            patient[col] = patient[col].dropna().iloc[-1]

    # print(patient)
    break

last_hour = pd.DataFrame(np.array(df.iloc[:, :].values,  dtype=float)[11::12], columns=df.columns[:])

#normalize each column except the first 2
last_hour2 = last_hour.iloc[:,2:]
last_hour2 = last_hour2.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

#apply the changes to the original
last_hour.iloc[:,2:] = last_hour2
last_hour


<ipython-input-14-2400c3f435fd>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient[col] = patient[col].dropna().iloc[-1]


,pid,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,1.0,14.0,0.223529,NaN,NaN,0.053398,NaN,0.666667,0.185792,0.475,...,NaN,1.000000,NaN,0.697674,0.153191,0.262821,NaN,NaN,0.331658,0.743243
1,10.0,12.0,0.658824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.959184,NaN,NaN,NaN,0.301282,NaN,NaN,0.502513,NaN
2,100.0,13.0,0.623529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.938776,NaN,NaN,NaN,0.544872,NaN,NaN,0.402010,NaN
3,1000.0,12.0,0.752941,0.275362,NaN,NaN,NaN,0.666667,NaN,NaN,...,NaN,0.897959,NaN,NaN,NaN,0.326923,NaN,NaN,0.447236,NaN
4,10000.0,12.0,0.717647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.979592,NaN,NaN,NaN,0.397436,NaN,NaN,0.482412,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18990,9993.0,12.0,0.764706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.000000,NaN,NaN,NaN,0.538462,NaN,NaN,0.326633,NaN
18991,9995.0,12.0,0.682353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.918367,NaN,NaN,NaN,0.230769,NaN,NaN,0.693467,NaN
18992,9996.0,18.0,0.447059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.000000,NaN,NaN,NaN,0.429487,NaN,NaN,0.296482,NaN
18993,9998.0,12.0,0.870588,NaN,NaN,NaN,NaN,0.555556,NaN,NaN,...,NaN,1.000000,NaN,NaN,NaN,0.461538,NaN,NaN,0.603015,NaN


## code to remove nan values (obsolete)

In [6]:
patients = pd.read_csv("train_features.csv")
# patients = pd.read_csv("test_features.csv")

patients.interpolate(method='linear', inplace=True)
#get the mean for each column
mean_values = patients.mean()


# go through 12 rows at a time
for i in range(0, len(patients), 12):
    # get the 12 rows
    patient = patients.iloc[i:i+12]
    #for each column, use linear interpolation to replace the nan values
    for col in patient.columns:
        patient[col] = patient[col].interpolate(method='linear')
        # mean = patient[col].mean()
        # if math.isnan(mean):
        #     patient[col] = mean_values[col]
        mean = patient[col].mean()
        patient[col] = patient[col].fillna(mean)



    if i%1000 == 0:
        print(i)
        # break
    # break



#save the dataframe
# patients.to_csv("train_features_fixed.csv", index=False, float_format='%.3f',na_rep='nan')
patients.to_csv("train_features_fixed_withnancols.csv", index=False, float_format='%.3f',na_rep='nan')
# patients.to_csv("test_features_fixed_withnancols.csv", index=False, float_format='%.3f',na_rep='nan')




<ipython-input-6-8cddb51b224d>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient[col] = patient[col].interpolate(method='linear')
<ipython-input-6-8cddb51b224d>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient[col] = patient[col].fillna(mean)


0
3000
6000
9000
12000
15000
18000
21000
24000
27000
30000
33000
36000
39000
42000
45000
48000
51000
54000
57000
60000
63000
66000
69000
72000
75000
78000
81000
84000
87000
90000
93000
96000
99000
102000
105000
108000
111000
114000
117000
120000
123000
126000
129000
132000
135000
138000
141000
144000
147000
150000
153000
156000
159000
162000
165000
168000
171000
174000
177000
180000
183000
186000
189000
192000
195000
198000
201000
204000
207000
210000
213000
216000
219000
222000
225000


# (obsolete)

In [7]:
# patients = pd.read_csv("train_features_fixed.csv")
# patients = pd.read_csv("test_features_fixed.csv")
patients = pd.read_csv("train_features_fixed_withnancols.csv")

#normalize each column except the first 2
patients2 = patients.iloc[:,2:]
patients2 = patients2.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

#apply the changes to the original
patients.iloc[:,2:] = patients2


#save the dataframe
patients.to_csv("train_features_normalized_withnancols.csv", index=False, float_format='%.3f',na_rep='nan')

In [23]:
# s = pd.Series([np.nan, 2, 4, 8,np.nan,12])
# s = s.interpolate(method='quadratic')
# s

0          NaN
1     2.000000
2     4.000000
3     8.000000
4    10.842105
5    12.000000
dtype: float64

In [81]:
rbf = svm.SVC(kernel="rbf",probability=True)
rbf.fit([[1,2],[1,5],[7,2]], [[0],[1],[0]])
pred = rbf.predict_proba([[-3,8]])
pred[:,1]

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


array([0.51689811])

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv("train_features_imputed_normalized.csv")
labels_df = pd.read_csv("train_labels.csv")
X = np.array(df.iloc[:, 1:].values,  dtype=float)
X_train = X[:int(X.shape[0]*0.8)]
X_valid = X[int(X.shape[0]*0.8):]
Y = np.array(labels_df.iloc[:, 1:].values,  dtype=float)
Y_train = Y[:int(Y.shape[0]*0.8)]
Y_valid = Y[int(Y.shape[0]*0.8):]



In [27]:
import xgboost as xgb
#Random Forests(TM) in XGBoost
clf_xgb = xgb.XGBRFRegressor(max_depth=10)
# clf_xgb = xgb.XGBClassifier(objective='binary:logistic', max_depth=3, n_estimators=300, learning_rate=0.05)
clf_xgb.fit(X_train, Y_train[:,10],verbose=True, eval_metric="auc", eval_set=[(X_valid, Y_valid[:,10])])


[0]	validation_0-auc:0.68636


XGBRFRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bytree=1, enable_categorical=False, gamma=0, gpu_id=-1,
               importance_type=None, interaction_constraints='',
               max_delta_step=0, max_depth=10, min_child_weight=1, missing=nan,
               monotone_constraints='()', n_estimators=100, n_jobs=6,
               num_parallel_tree=100, objective='reg:squarederror',
               predictor='auto', random_state=0, reg_alpha=0,
               scale_pos_weight=1, tree_method='exact', validate_parameters=1,
               verbosity=None)

In [28]:
pred = clf_xgb.predict(X_valid)
print(pred)
score = metrics.roc_auc_score(Y_valid[:,10], pred)
score

[0.04021233 0.0273331  0.10143197 ... 0.03193904 0.03128313 0.02972065]


0.686355791542457

In [32]:
import xgboost as xgb

# specify parameters via map
param = {'booster': 'dart',
         'max_depth': 5, 'learning_rate': 0.1,
         'objective': 'binary:logistic', 'silent': True,
         'sample_type': 'uniform',
         'normalize_type': 'tree',
         'rate_drop': 0.1,
         'skip_drop': 0.5}
num_round = 50
bst = xgb.train(param, xgb.DMatrix(X_train, label=Y_train[:,10]), num_round)
# make prediction
# ntree_limit must not be 0
preds = bst.predict(xgb.DMatrix(X_valid, label=Y_valid[:,10]))

[09:56:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:56:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [33]:
print(preds)
score = metrics.roc_auc_score(Y_valid[:,10], preds)
score

[0.03662579 0.040127   0.16152854 ... 0.04153312 0.04501774 0.03768276]


0.689327189599694

In [1]:
import xgboost as xgb
# # read in data
dtrain = xgb.DMatrix('train_features_imputed_normalized.csv')
dtest = xgb.DMatrix('train_labels_0.csv')
# specify parameters via map
param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
num_round = 2
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)
print(preds.shape)

In [3]:

model = ExtraTreesClassifier(n_estimators=100, random_state=0)#, probability=False
model.fit([[1,2,3],[4,2,3]], [0,1])


TypeError: __init__() got an unexpected keyword argument 'probability'

In [47]:
model.predict_proba([[2,2,3]])
# model.predict([[2,2,3]])

array([[0.62, 0.38]])

In [57]:
loss = tf.keras.losses.binary_crossentropy([[1]], [[0.1]])
loss.numpy()[0]


2.3025842

In [10]:

df = pd.read_csv("train_features.csv")
orig_ordering = pd.DataFrame(np.array(df.iloc[:, :].values,  dtype=float)[11::12], columns=df.columns[:])


#COUNT
count = pd.DataFrame(df.groupby(["pid"]).count())
count = pd.merge(orig_ordering["pid"], count, on="pid")
count = count.drop(["Time", "Age"], axis=1)
count.columns = [x+"_count" for x in count.columns]
count.rename(columns={'pid_count': 'pid'}, inplace=True)
print(type(count))
print(count.shape)

#MEAN
mean = pd.DataFrame(df.groupby(["pid"]).mean())
mean = pd.merge(orig_ordering["pid"], mean, on="pid")
mean = mean.drop(["Time", "Age"], axis=1)
mean.columns = [x+"_mean" for x in mean.columns]
mean.rename(columns={'pid_mean': 'pid'}, inplace=True)
print(type(mean))
print(mean.shape)
count

<class 'pandas.core.frame.DataFrame'>
(18995, 35)
<class 'pandas.core.frame.DataFrame'>
(18995, 35)


,pid,EtCO2_mean,PTT_mean,BUN_mean,Lactate_mean,Temp_mean,Hgb_mean,HCO3_mean,BaseExcess_mean,RRate_mean,...,Alkalinephos_mean,SpO2_mean,Bilirubin_direct_mean,Chloride_mean,Hct_mean,Heartrate_mean,Bilirubin_total_mean,TroponinI_mean,ABPs_mean,pH_mean
0,1.0,NaN,NaN,12.0,NaN,36.750000,8.566667,25.333333,-0.666667,17.000000,...,NaN,100.000000,NaN,112.0,23.200000,77.083333,NaN,NaN,114.500000,7.3700
1,10.0,NaN,27.80,12.0,NaN,36.000000,14.600000,NaN,NaN,18.090909,...,68.0,98.000000,NaN,NaN,42.100000,78.818182,1.30,0.010,132.909091,NaN
2,100.0,NaN,20.90,21.0,NaN,36.250000,12.500000,27.000000,NaN,14.833333,...,NaN,96.500000,NaN,101.0,36.800000,109.083333,NaN,NaN,117.000000,NaN
3,1000.0,31.863636,NaN,22.0,3.855,36.818182,9.200000,NaN,NaN,12.000000,...,NaN,98.818182,NaN,NaN,27.300000,86.363636,NaN,NaN,141.909091,7.3000
4,10000.0,NaN,28.55,22.0,NaN,36.750000,10.700000,25.500000,1.500000,12.090909,...,NaN,98.545455,NaN,103.5,30.300000,77.090909,NaN,NaN,123.000000,7.3900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18990,9993.0,NaN,NaN,13.5,NaN,35.750000,8.350000,NaN,NaN,19.125000,...,65.0,99.272727,NaN,NaN,26.150000,112.818182,0.55,10.680,99.636364,NaN
18991,9995.0,NaN,55.50,50.0,NaN,36.000000,11.200000,29.000000,NaN,19.545455,...,NaN,94.545455,NaN,89.0,35.600000,62.636364,NaN,NaN,180.181818,NaN
18992,9996.0,NaN,NaN,NaN,NaN,37.400000,10.800000,NaN,0.000000,16.500000,...,NaN,99.583333,NaN,NaN,30.033333,98.500000,NaN,NaN,99.250000,7.4025
18993,9998.0,NaN,34.70,13.0,NaN,36.636364,8.400000,NaN,NaN,20.454545,...,NaN,100.000000,NaN,NaN,25.600000,97.454545,NaN,1.505,131.545455,NaN
